In [2]:
import seaborn as sns
import random
import os
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import json 
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

%matplotlib inline

2024-06-27 12:14:14.612917: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
photos = pd.read_json("photos.json", lines = True)

In [4]:
photos1 = photos.drop(columns=['caption'])

In [5]:
photos2 = photos1.drop(columns=['business_id'])

In [6]:
def df_1(photo_id): #aplicamos un filtro para las imágenes corruptas del df
    try:
        Image.open(f"photos2/{photo_id}.jpg")
        return True
    except:
        return False

lista_images = photos2["photo_id"].to_list()
corrupt_images = []
for imagen in lista_images:
     if not df_1(imagen):
         corrupt_images.append(imagen)    

IGUALAR

In [7]:
def igualar1(photos2, etiqueta_columna, valor_columna, etiqueta_valor, num_filas):
    # Filtrar las filas que tienen la etiqueta específica
    filas_etiqueta = photos2[photos2[etiqueta_columna] == etiqueta_valor]
    
    # Verifica que num_filas no sea mayor que el total de filas con la etiqueta específica
    if num_filas > len(filas_etiqueta):
        raise ValueError("Error, coge un número más chico")
    
    # Seleccionar aleatoriamente los índices de las filas a eliminar
    indices_a_eliminar = np.random.choice(filas_etiqueta.index, num_filas, replace=False)
    
    # Eliminar las filas seleccionadas del dataset original
    photos3 = photos2.drop(indices_a_eliminar)
    
    return photos3 #photos3 se aplicará al df con la reducción de la etiqueta "food"

# Parámetros
etiqueta_columna = 'label'  # Nombre de la columna con la etiqueta
valor_columna = 'photo_id'       # Nombre de la columna con el valor asociado
etiqueta_valor = 'food'            # Valor específico de la etiqueta
num_filas = 106000                   # Número de filas a eliminar

# Eliminar filas con la etiqueta específica y su valor asociado
photos3 = igualar1(photos2, etiqueta_columna, valor_columna, etiqueta_valor, num_filas)

In [8]:
def igualar2(photos3, etiqueta_columna, valor_columna, etiqueta_valor, num_filas):
    
    filas_etiqueta = photos3[photos3[etiqueta_columna] == etiqueta_valor]
    
    
    if num_filas > len(filas_etiqueta):
        raise ValueError("Error, coge un número más chico")
    
    
    indices_a_eliminar = np.random.choice(filas_etiqueta.index, num_filas, replace=False)
    
    
    photos4 = photos3.drop(indices_a_eliminar)
    
    return photos4 #photos4 se aplicará al df con la reducción de la etiqueta "inside"

# Parámetros
etiqueta_columna = 'label'  # Nombre de la columna con la etiqueta
valor_columna = 'photo_id'       # Nombre de la columna con el valor asociado
etiqueta_valor = 'inside'            # Valor específico de la etiqueta
num_filas = 53802                  # Número de filas a eliminar


photos4 = igualar2(photos3, etiqueta_columna, valor_columna, etiqueta_valor, num_filas)

In [9]:
def igualar3(photos4, etiqueta_columna, valor_columna, etiqueta_valor, num_filas):
    
    filas_etiqueta = photos4[photos4[etiqueta_columna] == etiqueta_valor]
    
    
    if num_filas > len(filas_etiqueta):
        raise ValueError("Error, coge un número más chico")
    
    
    indices_a_eliminar = np.random.choice(filas_etiqueta.index, num_filas, replace=False)
    
    
    photos5 = photos4.drop(indices_a_eliminar)
    
    return photos5 #photos5 se aplicará al df con la reducción de la etiqueta "outside"

# Parámetros
etiqueta_columna = 'label'  # Nombre de la columna con la etiqueta
valor_columna = 'photo_id'       # Nombre de la columna con el valor asociado
etiqueta_valor = 'outside'            # Valor específico de la etiqueta
num_filas = 16101                  # Número de filas a eliminar


photos5 = igualar3(photos4, etiqueta_columna, valor_columna, etiqueta_valor, num_filas)

In [10]:
def igualar4(photos5, etiqueta_columna, valor_columna, etiqueta_valor, num_filas):
    
    filas_etiqueta = photos5[photos5[etiqueta_columna] == etiqueta_valor]
    
    
    if num_filas > len(filas_etiqueta):
        raise ValueError("Error, coge un número más chico")
    
    
    indices_a_eliminar = np.random.choice(filas_etiqueta.index, num_filas, replace=False)
    
    
    photos6 = photos5.drop(indices_a_eliminar)
    
    return photos6 #photos6 se aplicará al df con la reducción de la etiqueta "drink"

# Parámetros
etiqueta_columna = 'label'  # Nombre de la columna con la etiqueta
valor_columna = 'photo_id'       # Nombre de la columna con el valor asociado
etiqueta_valor = 'drink'            # Valor específico de la etiqueta
num_filas = 13020                 # Número de filas a eliminar


photos6 = igualar4(photos5, etiqueta_columna, valor_columna, etiqueta_valor, num_filas)

LIMPIO 

In [17]:
df_photos = photos6.copy()

In [18]:
df_photos.head()

,photo_id,label
16,6LJa8QZOGBjt9maE5miLhw,menu
69,DtsfhqhqUB-tjs5XbwJfXw,menu
72,9Wkj-Vl5EYLS6OIy4EYigw,food
82,RXtykJb7KIXHySXJJnlFfg,inside
94,pRnGI7xv7ZBJ8p_w0xy9fg,drink


In [20]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os

In [24]:
# Ruta a la carpeta de imágenes
image_folder = '/Users/florentinoromerourquiza/Documents/yelp_photos-002/photos'  # Asegúrate de ajustar la ruta

# Añadir una columna con las rutas completas de las imágenes
df_photos['image_path'] = df_photos['photo_id'].apply(lambda x: os.path.join(image_folder, x))

# Verificar si las rutas de las imágenes existen
assert df_photos['image_path'].apply(os.path.exists).all(), "Some image paths do not exist"

# Codificar las etiquetas
le = LabelEncoder()
df_photos['label_encoded'] = le.fit_transform(df_photos['label'])

# Convertir las etiquetas codificadas a cadenas
df_photos['label_encoded'] = df_photos['label_encoded'].astype(str)

# Dividir los datos en conjuntos de entrenamiento y validación
train_df, val_df = train_test_split(df_photos, test_size=0.2, stratify=df_photos['label_encoded'], random_state=42)

# Verificar si los DataFrames no están vacíos
assert not train_df.empty, "Training DataFrame is empty"
assert not val_df.empty, "Validation DataFrame is empty"

# Parámetros
img_height, img_width = 224, 224
batch_size = 32
num_classes = len(df_photos['label'].unique())

# Generadores de datos con aumento de imagen
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='image_path',
    y_col='label_encoded',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='image_path',
    y_col='label_encoded',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Cargar MobileNetV2 sin las capas superiores (headless)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Congelar las capas base
base_model.trainable = False

# Añadir nuestras propias capas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Crear el modelo completo
model = Model(inputs=base_model.input, outputs=predictions)

# Compilar el modelo
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Evaluación y cálculo del F1-score
val_generator.reset()
preds = model.predict(val_generator, steps=val_generator.n // batch_size + 1)
pred_labels = np.argmax(preds, axis=1)
true_labels = val_df['label_encoded'].astype(int).values

# Calcular F1-score
f1 = f1_score(true_labels, pred_labels, average='weighted')
print(f"F1-score: {f1}")

AssertionError: Some image paths do not exist